In [2]:
# Import libraries and packages
import matplotlib as plt
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from keras.models import Sequential
from keras.layers.core import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import layers, losses
#from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import random
import pickle
import cv2
import os
import time   # time1 = time.time(); print('Time taken: {:.1f} seconds'.format(time.time() - time1))
import warnings
warnings.filterwarnings("ignore")
SEED = 42   # set random seed

In [3]:
def read_data():
    """
    Reads Image Files from Current Working Directory
    and returns A list of paths to each image and their CLass names
    
    """
    root_dir = os.path.join(os.getcwd(), "Data\\")
    class_names = []
    paths = []
    print("reading images from folders")
    for folder in os.listdir(root_dir):
        subdir = os.path.join(root_dir, f"{folder}\\")
        print(f"{folder}--Folder")
        for eachfile in os.listdir(subdir):
            class_names.append(int(folder))
            image = cv2.imread(os.path.join(subdir, f"{eachfile}"))
            image = cv2.resize(image, (100, 100)).flatten()
            paths.append(image)
    print("Done loading Images....") 
    data = np.array(paths, dtype="uint8") / 255.0
    labels = np.array(class_names)
    return data, labels


## Load data from disk

In [4]:
data, labels = read_data()

reading images from folders
Done loading Images....


## Split data for training and validation

In [5]:
# partition the data into 80% training and 20% validation
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.2, random_state=SEED)

In [6]:
trainX.shape

(7220, 30000)

In [7]:
trainY

array([ 6,  4, 10, ...,  2,  1,  6])

In [8]:
# convert the labels from integers/categories to vectors 
# (for 2-class, binary classification you should use Keras' to_categorical function instead)
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)   # fit_transform = find all unique class labels + transform into one-hot encoded labels
testY = lb.transform(testY)         # transform = perform the one-hot encoding (unique class labels already found)

# This is the categorical vector after transformation
# [1, 0, 0] # corresponds to cat
# [0, 1, 0] # corresponds to dog
# [0, 0, 1] # corresponds to panda

In [9]:
trainY

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

## Define neural network architecture using Keras

In [10]:
# define the 3072-1024-512-3 architecture using Keras

model = Sequential()   # input layer 30000 as there are 100x100x3=30000 pixels in a flattened input image
model.add(Dense(1024, input_shape=(30000,), activation="sigmoid"))   # first hidden layer has 1024 nodes
model.add(Dense(512, activation="sigmoid"))                         # second hidden layer has 512 nodes
model.add(Dense(len(lb.classes_), activation="softmax"))            # output layer with number of possible class labels

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1024)              30721024  
                                                                 
 dense_1 (Dense)             (None, 512)               524800    
                                                                 
 dense_2 (Dense)             (None, 21)                10773     
                                                                 
Total params: 31,256,597
Trainable params: 31,256,597
Non-trainable params: 0
_________________________________________________________________


## Compile Model

In [12]:
# initialize our initial learning rate and # of epochs to train for
INIT_LR = 0.01
EPOCHS = 60
 
# compile the model using SGD as our optimizer and categorical cross-entropy loss
# (you'll want to use binary_crossentropy for 2-class classification)
print("[INFO] training network...")
opt = SGD(lr=INIT_LR)   # Stochastic Gradient Descent (SGD) optimizer
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

[INFO] training network...


## Fit (ie, Train) model

In [ ]:
# train the neural network on training data set
# batch_size (32) controls the size of each group of data to pass through the network. 
# Larger GPUs would be able to accommodate larger batch sizes (eg. 64)
time1 = time.time()   # to measure time taken
H = model.fit(trainX, trainY, validation_data=(testX, testY), epochs=EPOCHS, batch_size=32)
print('Time taken: {:.1f} seconds'.format(time.time() - time1))   # to measure time taken

Epoch 1/60
226/226 [==============================] - 264s 355ms/step - loss: 2.9222 - accuracy: 0.1346 - val_loss: 2.8413 - val_accuracy: 0.1451
Epoch 2/60
226/226 [==============================] - 64s 281ms/step - loss: 2.7443 - accuracy: 0.2019 - val_loss: 2.6663 - val_accuracy: 0.2270
Epoch 3/60
226/226 [==============================] - 63s 280ms/step - loss: 2.5809 - accuracy: 0.2619 - val_loss: 2.5303 - val_accuracy: 0.2890
Epoch 4/60
160/226 [====================>.........] - ETA: 15s - loss: 2.4473 - accuracy: 0.3010

## Evaluate Model

In [ ]:
# evaluate the network
print("[INFO] evaluating Artificial Neural Network Using Test Data...")
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1)))#, target_names=lb.classes_))

In [ ]:
# evaluate the network
print("[Classification Report] evaluating Artificial Neural Network Using Training Data...")
predictions = model.predict(trainX, batch_size=32)
print(classification_report(trainY.argmax(axis=1), predictions.argmax(axis=1)))

In [ ]:
# evaluate the network
print("[Confusion Matrix] evaluating Artificial Neural Network Using Training Data...")
predictions = model.predict(trainX, batch_size=32)
c_mat = confusion_matrix(trainY.argmax(axis=1), predictions.argmax(axis=1))
ConfusionMatrixDisplay(confusion_matrix= c_mat).plot()#, target_names=lb.classes_))
plt.savefig("ConfusionMatrix_train(ANN).jpg")
plt.show()

In [ ]:
# evaluate the network
print("[Confusion Matrix] evaluating Artificial Neural Network Using Testing Data...")
predictions = model.predict(testX, batch_size=32)
c_mat = confusion_matrix(testY.argmax(axis=1), predictions.argmax(axis=1))
ConfusionMatrixDisplay(confusion_matrix= c_mat).plot()#, target_names=lb.classes_))
plt.savefig("ConfusionMatrix_test(ANN).jpg")
plt.show()

In [ ]:
# evaluate the network
print("[Classification Report] evaluating Artificial Neural Network Using Testing Data...")
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1)))

In [ ]:
# plot the training and validation loss
N = np.arange(0, EPOCHS)
plt.style.use("ggplot")
plt.figure(figsize = [10,8])
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.title("Training & Validation Loss (Artificial NN)")
plt.xlabel("Epoch #", weight="bold")
plt.ylabel("Loss", weight="bold")
plt.legend()
plt.savefig("trainingdataloss(ANN).png")
plt.show()

In [ ]:
# plot the training and validation accuracy
N = np.arange(0, EPOCHS)
plt.style.use("ggplot")
plt.figure(figsize = [10,8])
plt.plot(N, H.history["accuracy"], label="train_acc")
plt.plot(N, H.history["val_accuracy"], label="val_acc")
plt.title("Training and Validation Accuracy (Artificial NN)")
plt.xlabel("Epoch #", weight="bold")
plt.ylabel("Accuracy", weight="bold")
plt.legend()
plt.savefig("trainingdataAccuracy(ANN).png")
plt.show()

## Save model

In [ ]:
model.save("./models/ANNModel")